In [17]:
import os

import scenedetect
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.frame_timecode import FrameTimecode
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors import ContentDetector

STATS_FILE_PATH = 'testvideo.stats.csv'

# Create a video_manager point to video file testvideo.mp4. Note that multiple
# videos can be appended by simply specifying more file paths in the list
# passed to the VideoManager constructor. Note that appending multiple videos
# requires that they all have the same frame size, and optionally, framerate.
video_manager = VideoManager(['goldeneye.mp4'])
stats_manager = StatsManager()
scene_manager = SceneManager(stats_manager)
# Add ContentDetector algorithm (constructor takes detector options like threshold).
scene_manager.add_detector(ContentDetector())
base_timecode = video_manager.get_base_timecode()

try:
    # If stats file exists, load it.
    if os.path.exists(STATS_FILE_PATH):
        # Read stats from CSV file opened in read mode:
        with open(STATS_FILE_PATH, 'r') as stats_file:
            stats_manager.load_from_csv(stats_file, base_timecode)

    start_time = base_timecode + 20     # 00:00:00.667
    end_time = base_timecode + 20.0     # 00:00:20.000
    # Set video_manager duration to read frames from 00:00:00 to 00:00:20.
    video_manager.set_duration(start_time=start_time, end_time=end_time)

    # Set downscale factor to improve processing speed.
    video_manager.set_downscale_factor()

    # Start video_manager.
    video_manager.start()

    # Perform scene detection on video_manager.
    scene_manager.detect_scenes(frame_source=video_manager)

    # Obtain list of detected scenes.
    scene_list = scene_manager.get_scene_list(base_timecode)
    # Like FrameTimecodes, each scene in the scene_list can be sorted if the
    # list of scenes becomes unsorted.

    print('List of scenes obtained:')
    for i, scene in enumerate(scene_list):
        print('    Scene %2d: Start %s / Frame %d, End %s / Frame %d' % (
            i+1,
            scene[0].get_timecode(), scene[0].get_frames(),
            scene[1].get_timecode(), scene[1].get_frames(),))
        
    # We only write to the stats file if a save is required:
    if stats_manager.is_save_required():
        with open(STATS_FILE_PATH, 'w') as stats_file:
            stats_manager.save_to_csv(stats_file, base_timecode)

finally:
    video_manager.release()

100%|██████████| 460/460 [00:00<00:00, 782.86frames/s]

List of scenes obtained:
    Scene  1: Start 00:00:00.834 / Frame 20, End 00:00:03.754 / Frame 90
    Scene  2: Start 00:00:03.754 / Frame 90, End 00:00:08.759 / Frame 210
    Scene  3: Start 00:00:08.759 / Frame 210, End 00:00:10.802 / Frame 259
    Scene  4: Start 00:00:10.802 / Frame 259, End 00:00:15.599 / Frame 374
    Scene  5: Start 00:00:15.599 / Frame 374, End 00:00:19.186 / Frame 460


In [10]:
scenedetect.video_splitter.is_ffmpeg_available()

True

In [15]:
scenedetect.video_splitter.split_video_ffmpeg(['goldeneye.mp4'], 'testvideo.stats.csv', '-scene', 'xxx', arg_override='-c:v libx264 -preset fast -crf 21 -c:a copy', hide_progress=False, suppress_output=False)

IndexError: string index out of range